In [12]:
import smtplib
import email
from email.MIMEMultipart import MIMEMultipart
from email.MIMEText import MIMEText
from email.MIMEBase import MIMEBase
from email import Encoders
import keyring as k
import datetime
import FiOps_Functions as Funcs
import os
import pandas as pd
from datetime import datetime 

In [13]:
username = 'jjaros'
conn = Funcs.db_connect(username)



Please enter your Database Password: 
Note: Please copy and paste your password, by right click pasting. Ctrl - V does not work in the Command Prompt


In [ ]:
last_refreshed_query = """ 
select bi.EXECUTED from bi_private.etl_job_status bi
WHERE bi.JOB = 'Skyrocket_Lead_Disposition_Reporting'
AND bi.STATUS IN ('PASS','SUCCESS')
ORDER BY bi.EXECUTED DESC LIMIT 1
"""
Last_Refreshed_Date = pd.read_sql_query(last_refreshed_query, conn)
Last_Refreshed_Date = Last_Refreshed_Date.loc[0, 'Last_Refreshed_Date']
Last_Refreshed_Date = datetime.strftime(Last_Refreshed_Date,"%Y-%m-%d %H:%M:%S")
Last_Refreshed_Date


In [44]:
today = datetime.today().strftime("%Y.%m.%d")
today
mytime = datetime.strptime('0830','%H%M').time()
email_send = datetime.combine(datetime.today().date(), mytime)
email_send

datetime.datetime(2017, 2, 28, 8, 30)

In [40]:
if Last_Refreshed_Date < email_send:
    
    

IndentationError: expected an indented block (<ipython-input-40-8d303a6c6126>, line 2)

In [16]:
raw_data_query = """
select
    data.`Lead` as 'Lead Count',
    data.`Qualification` as 'Qualification',
	data.`Gross Sale` as 'Gross Sale',
    data.`State Final` as 'State/Province',
    
    case 
        
-- --------------------------------------- after 10/1/16 re-negotiation --------------------------------------------------------------------------	
    
    when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('CA') and data.`iQuote Request Date` >='2016-10-01')
		then 280     
	when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('NJ','RI') and data.`iQuote Request Date` >='2016-10-01')
		then 200
	when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('MA') and data.`iQuote Request Date` >='2016-10-01')
		then 238 
 	when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('CT','VT') and data.`iQuote Request Date` >='2016-10-01')
		then 200
    when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('NY') and data.`iQuote Request Date` >='2016-10-01') 
		then 165        
    else 0
    END as 'ALS Lead/Qual/Sale Price',
    case when data.`Referral Code Provided in Funnel Final` = '9252322354'
		then '9252322354' 
	when data.`Referral Code Provided in Funnel Final` = 'SKYROCKET'
		then 'SKYROCKET'
    when data.`Referral Code Final` = '9252322354'
		then '9252322354'
	when data.`Referral Code Final` = 'SKYROCKET'
		then 'SKYROCKET'
	ELSE null
    END
	as 'Referral Code',
data.`iQuote Request Date` as 'Created Date',
    Date(data.`Qualification Date`) as 'Qualification Date',
    data.`Zip Code` as 'Zip Code',
    data.`Utility Final` as 'Utility',
    data.`Stage` as 'Opportunity Stage',
    data.`Stage Detail` as 'Opportunity Stage Detail',
	L.LeadSource as 'Lead Source',
	L.Email as 'Email',
    COALESCE(L.Phone, C.Phone, C.MOBILEPHONE, C.WORK_PHONE__C) as 'Phone Number',
    data.`Lead Created Date Time` as 'Lead Created Date Time',
    data.`Sale Date`,
    data.`Lead` as 'Sungevity Lead Count'
    
from sales_analytics.data 
left join salesforce_prod.Lead L on L.ID = data.Lead__c
left join salesforce_prod.opportunity o on o.ID = data.Opportunity__c
left join salesforce_prod.account a on a.ID = data.Account__c
left join salesforce_prod.accountcontactrole ACR on ACR.ACCOUNTID = A.ID
     and ACR.ROLE like 'Decision Maker/Borrower'
left join salesforce_prod.contact c on c.ID = ACR.CONTACTID

where

(Year(data.`Sale Date`) = year(CurDate())
OR Year(Data.`iQuote Request Date`)=Year(CurDate()))
and (data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = 'SKYROCKET'
		OR data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final` = '9252322354')
"""

In [17]:
summary_data_query = """
select
concat(monthname(y.date), '-',year(y.date)) as Month,
concat('$',format(sum(leads.cost),0)) as 'Cost',
sum(leads.Leads) as 'Leads',
concat('$',format(sum(leads.cost)/sum(leads.leads),2)) as 'Cost Per Lead',
sum(leads.Qualification) as 'Opportunities',
concat('$',format(sum(leads.cost)/sum(leads.qualification),0)) as 'Cost per Opp',
concat(format(sum(leads.qualification)/sum(leads.leads)*100,2),'%') as 'Qual Rate',
sum(sales.Sales) as 'Gross Sales',
concat(format(sum(sales.Sales)/sum(leads.Qualification)*100,2),'%') as 'Close Rate',
concat(format(sum(sales.sales)/sum(leads.leads)*100,2),'%') as 'Conv Rate',
concat('$',format(sum(leads.cost)/sum(sales.sales),2)) as 'CAC'


from marketing.yearly_calendar Y
left join
	(
    select
    
    sum(data.Lead) as 'Leads',
    data.`iQuote Request Date`, 
    sum(data.`Qualification`) as 'Qualification',
    sum(case 
        
-- --------------------------------------- after 10/1/16 re-negotiation --------------------------------------------------------------------------	
    
    when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('CA') and data.`iQuote Request Date` >='2016-10-01')
		then 280     
	when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('NJ','RI') and data.`iQuote Request Date` >='2016-10-01')
		then 200
	when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('MA') and data.`iQuote Request Date` >='2016-10-01')
		then 238 
 	when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('CT','VT') and data.`iQuote Request Date` >='2016-10-01')
		then 200
    when ((data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final`='SKYROCKET'
			or data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = '9252322354') and data.`State Final` in ('NY') and data.`iQuote Request Date` >='2016-10-01') 
		then 165
 -- ----------------------------------------- end of skyrocket ---------------------------------------------------------------------------------    
        
    else 0
    END )as 'Cost'
    
    
    from sales_analytics.data 
	left join salesforce_prod.Lead L on L.ID = data.Lead__c
	left join salesforce_prod.opportunity o on o.ID = data.Opportunity__c
	left join salesforce_prod.account a on a.ID = data.Account__c
	left join salesforce_prod.accountcontactrole ACR on ACR.ACCOUNTID = A.ID
		 and ACR.ROLE like 'Decision Maker/Borrower'
	left join salesforce_prod.contact c on c.ID = ACR.CONTACTID

	where

	(Year(data.`iQuote Request Date`) = Year(curdate()))
	and (data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = 'SKYROCKET'
			OR data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final` = '9252322354')
	group by data.`iQuote Request Date`
            
	)leads on leads.`iQuote Request Date` = Y.date
left join 
	(
    select
    sum(data.`Gross Sale`) as 'Sales',
    data.`Sale Date`
    
    from sales_analytics.data 
	left join salesforce_prod.Lead L on L.ID = data.Lead__c
	left join salesforce_prod.opportunity o on o.ID = data.Opportunity__c
	left join salesforce_prod.account a on a.ID = data.Account__c
	left join salesforce_prod.accountcontactrole ACR on ACR.ACCOUNTID = A.ID
		 and ACR.ROLE like 'Decision Maker/Borrower'
	left join salesforce_prod.contact c on c.ID = ACR.CONTACTID

	where

	(Year(data.`Sale Date`) = year(curDate()))
	and (data.`Referral Code Final` = '9252322354' or data.`Referral Code Provided in Funnel Final` = 'SKYROCKET'
			OR data.`Referral Code Final` = 'SKYROCKET' or data.`Referral Code Provided in Funnel Final` = '9252322354')
	group by data.`Sale Date`
	)sales on sales.`Sale Date` = Y.date

where year(Y.Date) = year(curdate()) and Y.Date <= Curdate()

Group by concat(monthname(y.date), '-',year(y.date))
ORDER BY MONTH(Y.DATE)
"""

In [18]:
data = pd.read_sql_query(raw_data_query,conn)
summary = pd.read_sql_query(summary_data_query, conn)

In [19]:
data

,Lead Count,Qualification,Gross Sale,State/Province,ALS Lead/Qual/Sale Price,Referral Code,Created Date,Qualification Date,Zip Code,Utility,Opportunity Stage,Opportunity Stage Detail,Lead Source,Email,Phone Number,Lead Created Date Time,Sale Date,Sungevity Lead Count
0,1,1,1,CA,0,SKYROCKET,2016-08-29,2016-09-05,94708,PGE,Sold Compliant,None,269739_4212,maxfratoni@gmail.com,(510) 384-5295,2016-08-29 07:24:14,2017-02-16,1
1,1,1,1,RI,200,SKYROCKET,2016-10-07,2016-10-08,02871,National Grid - RI,Cancelled,None,269739_6415,susancrockett50828@yahoo.com,4018470776,2016-10-07 15:55:24,2017-01-19,1
2,1,1,1,CT,200,SKYROCKET,2016-10-13,2016-10-13,06375,Eversource - CT,Sold Compliant,None,269739_6733,stephen.roy@snet.net,(860) 442-9288,2016-10-13 10:47:50,2017-02-24,1
3,1,1,1,MA,238,9252322354,2016-11-14,2016-11-18,02127,Eversource - MA (NSTAR),Cancelled,None,None,mike@caplanpropertiesboston.com,(508) 930-8932,2016-11-14 10:30:39,2017-01-09,1
4,1,1,1,CA,280,SKYROCKET,2016-11-19,2016-11-19,93274,SCE,Closed Won,None,269739_9938,tulerue@gmail.com,(559) 686-6836,2016-11-19 12:50:02,2017-01-29,1
5,1,1,1,MA,238,SKYROCKET,2016-11-29,2016-11-29,02053,Eversource - MA (NSTAR),Cancelled,None,269739_10399,dbliss7@hotmail.com,(508) 321-1891,2016-11-29 10:55:55,2017-01-05,1
6,1,1,1,CA,280,SKYROCKET,2016-12-04,2016-12-06,93960,PGE,Sold Compliant,None,269752_10654,jessicasoul@sbcglobal.net,(831) 678-2015,2016-12-04 10:12:31,2017-01-03,1
7,1,1,1,CA,280,SKYROCKET,2016-12-12,2016-12-12,91401,LADWP,Closed Won,None,269739_11064,corey@c-cooper.com,(818) 535-1979,2016-12-12 12:17:30,2017-01-06,1
8,1,1,1,NY,165,SKYROCKET,2016-12-15,2016-12-15,10573,ConEd (Consolidated Edison New York),Sold Compliant,None,269739_11195,christopherqso@yahoo.com,(646) 873-0210,2016-12-15 16:55:24,2017-01-30,1
9,1,1,1,CA,280,SKYROCKET,2016-12-11,2016-12-20,92024,SDGE,Sold Compliant,None,269739_11025,rdowdyhome@gmail.com,(619) 972-8344,2016-12-11 13:59:21,2017-01-12,1


In [20]:
summary

,Month,Cost,Leads,Cost Per Lead,Opportunities,Cost per Opp,Qual Rate,Gross Sales,Close Rate,Conv Rate,CAC
0,January-2017,"$50,821",203.0,$250.35,119.0,$427,58.62%,35.0,29.41%,17.24%,"$1,452.03"
1,February-2017,"$113,983",447.0,$255.00,262.0,$435,58.61%,64.0,24.43%,14.32%,"$1,780.98"


In [21]:
report_name = today + '_Lead Disposition Report.xlsx'
userhome = os.path.expanduser('~/Desktop')
filename = userhome + '/' + report_name
filename

'C:\\Users\\nseelam/Desktop/2017.02.28_Lead Disposition Report.xlsx'

In [27]:
writer = pd.ExcelWriter(filename, engine='xlsxwriter',options={'encoding':'utf-8'}, date_format='mm/dd/yyyy', datetime_format = 'mm/dd/yyyy')
workbook = writer.book
worksheets = ['Summary','Data']
data.to_excel(writer,sheet_name='Data', index=False)
summary.to_excel(writer, sheet_name = 'Summary', index = False)
writer.save()

In [23]:
HEADER = '''
<html>
    <head>

    </head>
    <body>
'''
FOOTER = '''
    </body>
</html>
'''

with open('test.html', 'w') as f:
    f.write(HEADER)
    f.write(summary.to_html(classes='df'))
    f.write(FOOTER)

In [25]:
###################### Assigning Recipients and Sender of Email ##################

fromaddr = "jjaros@sungevity.com"
toaddr = ['jjaros@sungevity.com','nseelam@sungevity.com']
msg = MIMEMultipart()
msg['From'] = fromaddr
msg['To'] = ", ".join(toaddr)
msg['Subject'] = "[Skyrocket Reporting:] Sungevity // Skyrocket :: Lead Disposition Reporting - " + today

body = "Hi Ryan, \n\nPlease find the most recent Lead Disposition Report attached.\n"
msg.attach(MIMEText(body,'plain'))

################## Building the Contents of the Email #########################

filename_html = 'test.html'
f = file(filename_html)
attachment = MIMEText(f.read(), 'html')
msg.attach(attachment)

body = "\nBest, \n\nJasmine Jaros"
msg.attach(MIMEText(body,'plain'))

part = MIMEBase('application', "octet-stream")
part.set_payload( open(filename,"rb").read() )
Encoders.encode_base64(part)
part.add_header('Content-Disposition', 'attachment; filename="%s"' % report_name)
msg.attach(part)


################### Sending the email #########################

server = smtplib.SMTP('smtp.gmail.com', 587)
server.ehlo()
server.starttls()
server.ehlo()
server.login("jjaros@sungevity.com", 'Kangor24$')
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)


{}

In [28]:
os.remove(filename)